# Performing Catchment Delineation using Esri Ready-To-Use Tools

With an ArcGIS Online account or an ArcGIS Online Organization, users can use certain ready-to-use tools to perform geoprocessing against curated datasets such as elevation-derived hydrography. one such service is the [Watershed](https://developers.arcgis.com/rest/elevation/api-reference/watershed.htm) service, which uses best-available elevation data to perform wateshed delineation from points.

### Import modules

In [40]:
# Find the directory of the current project and add to PATH
import sys, os, arcpy
home_folder = arcpy.mp.ArcGISProject("current").homeFolder
sys.path.insert(0, home_folder)
os.chdir(home_folder)

# The 00_environment_setup notebook contains libraries and other things common to all the notebooks (e.g. file paths)
%run "00_environment_setup.ipynb"

Active Portal in ArcGIS Pro
Logged in as ksampson
Current conda environment:
	arcgispro-py3-clone C:\Users\ksampson\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone
Found input data directory: C:\Users\ksampson\Documents\GitHub\GloFAS_Q2Q_Bias_Correction_and_Verification\data\input
Completed importing and/or installing libraries in 4.58 seconds.


### Lets use the GloFAS station details to identify points of interest

In [41]:
station_details_URL = 'https://staff.ral.ucar.edu/hopson/GloFAS/Q2Qbiascorrection/glofas_stations_details.CSV'
    
# Create a data frame
df = pd.read_csv(station_details_URL)

# Subset the dataframe to desired stations
#df = df[df['sta_name'].isin(['007-CDJAIPUR', '019-LYDAGRA'])]

# Create a spatially enabled data frame
sedf = pd.DataFrame.spatial.from_xy(df=df, x_column='lon_cwc', y_column='lat_cwc', sr=4326)

# Check head for SHAPE column
sedf

,sta_name,sta_id,lon_cwc,lat_cwc,subbasin,basin,sta_name2,state,lon_snap,lat_snap,notes,SHAPE
0,mathanguri,001-LBDJPG,90.956,26.781,Manas,Brahmaputra,Mathanguri,Assam,90.956,26.781,CWC website,"{""spatialReference"": {""wkid"": 4326}, ""x"": 90.9..."
1,bedepara,001-lgd3beh,88.697,24.139,LowerGanges,Ganges,Bedepara,West Bengal,88.750,24.200,visual snapped,"{""spatialReference"": {""wkid"": 4326}, ""x"": 88.6..."
2,guwahati,001-MBDGHY,91.757,26.198,LowerBrahmaputra,Brahmaputra,Guwahati,Assam,91.757,26.198,CWC website,"{""spatialReference"": {""wkid"": 4326}, ""x"": 91.7..."
3,buxar,001-MGD5PTN,83.972,25.580,MiddleGanges,Ganges,Buxar,Bihar,83.989,25.588,visual snapped,"{""spatialReference"": {""wkid"": 4326}, ""x"": 83.9..."
4,china bridge,001NEID3,92.007,27.640,Manas,Brahmaputra,Tawang Chu (Muruga Bridge),NaN,92.007,27.640,visual snapped,"{""spatialReference"": {""wkid"": 4326}, ""x"": 92.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
61,ghugumari,030-LBDJPG,89.464,26.305,Jamuna,Brahmaputra,Ghugumari,West Bengal,89.464,26.305,CWC website,"{""spatialReference"": {""wkid"": 4326}, ""x"": 89.4..."
62,dhengra ghat,030-mgd4ptn,87.803,25.848,Mahananda,Ganges,Dhengra Ghat,Bihar,87.803,25.848,CWC website,"{""spatialReference"": {""wkid"": 4326}, ""x"": 87.8..."
63,jhawa,031-mgd4ptn,87.791,25.610,Mahananda,Ganges,Jhawa,Bihar,87.791,25.610,CWC website,"{""spatialReference"": {""wkid"": 4326}, ""x"": 87.7..."
64,tezpur,031-UBDDIB,92.798,26.627,LowerBrahmaputra,Brahmaputra,Tezpur,Assam,92.798,26.627,CWC website,"{""spatialReference"": {""wkid"": 4326}, ""x"": 92.7..."


### Perform point delineation using the Watershed Ready-To-Use tool and user-defined points

First, we will examine the tool help information by calling `help(<tool name>)` in the codeblock below.

In [42]:
help(arcpy.agolservices.Watershed)

Help on function Watershed in module arcpy.agolservices:

Watershed(InputPoints=None, PointIDField=None, SnapDistance=None, SnapDistanceUnits=None, DataSourceResolution=None, Generalize=None, ReturnSnappedPoints=None)
    Watershed_agolservices(InputPoints, {PointIDField}, {SnapDistance}, {SnapDistanceUnits}, {DataSourceResolution}, {Generalize}, {ReturnSnappedPoints})
    
       Determines the contributing area above each input point. A watershed
       is the upslope area that contributes flow.
    
    INPUTS:
     InputPoints (Feature Set):
         The point features used for calculating watersheds. These are referred
         to as pour points, because it is the location at which water pours out
         of the watershed.
     PointIDField {String}:
         An integer or string field used to identify to the input points.The
         default is to use the unique ID field.
     SnapDistance {String}:
         The maximum distance to move the location of an input
         point.In

The [Watershed](https://developers.arcgis.com/rest/elevation/api-reference/watershed.htm) takes a FeatureSet of points as input, which can be defined dynamically on the map, or from existing data. We will convert points from our Spatially-Enabled DataFrame into a FeatureSet to submit them for watershed delineation.

### Use the ready-to-use services available in the `arcpy.agolservices` module to perform station point delineation

Warning: this took 10 minutes in testing

In [ ]:
%%time

result = arcpy.agolservices.Watershed(
    InputPoints=sedf,
    PointIDField="",
    SnapDistance="500",
    SnapDistanceUnits="Meters",
    DataSourceResolution="90m",
    Generalize=True,
    ReturnSnappedPoints=True
)

In [ ]:
for n,in_path in enumerate(result.split('; ')):
    out_file = os.path.join(output_data_dir, 'result_{0}.shp'.format(n))
    arcpy.CopyFeatures_management(in_path, out_file)
    print(f'Saved input file {in_path} to {out_file}'.format())

### Reset the namespace

The following `%reset -f` command is a built-in command in Jupyter Notebook that will reset the namespace. This is good practice to run when you are finished with the notebook.

In [ ]:
%reset -f